In [3]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

c:\Users\marco\Learning\pybit-trading-bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
INPUT_WINDOW = 64

In [5]:
data = pd.read_csv("SOLUSDT-15m-180801.csv")

rate_of_change_1 = ((data["close"] - data["open"]) / data["open"]).to_numpy()[:-127]  # 15 Minutes
rate_of_change_4 = ((data["close"].shift(-3) - data["open"]) / data["open"]).to_numpy()[:-127]  # 1 Hour
rate_of_change_32 = ((data["close"].shift(-31) - data["open"]) / data["open"]).to_numpy()[:-127]  # 8 Hours
rate_of_change_128 = ((data["close"].shift(-127) - data["open"]) / data["open"]).to_numpy()[:-127]  # 1 Day

X = rate_of_change_4[:-1]
Y = rate_of_change_4[INPUT_WINDOW:]

X = np.lib.stride_tricks.sliding_window_view(X, INPUT_WINDOW)

X.shape, Y.shape

((98270, 64), (98270,))

In [16]:
import torch
from torch import nn
import torch.functional as F
from torch.utils.data import TensorDataset, DataLoader

X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

Xtrain = X[:int(len(X) * 0.8)]
Ytrain = Y[:int(len(Y) * 0.8)]
Xtest = X[int(len(X) * 0.8):]
Ytest = Y[int(len(Y) * 0.8):]

dataset = TensorDataset(Xtrain, Ytrain)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)


class TradingNN(nn.Module):

    def __init__(self):
        super(TradingNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 16)
        self.conv2 = nn.Conv1d(32, 64, 16)

        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(0.5)

        self.pool = nn.AvgPool1d(2)

    def forward(self, x):
        x = x.unsqueeze(1)

        x = self.conv1(x)
        x = self.dropout(x)
        x = self.pool(torch.relu(x))

        x = self.conv2(x)
        x = self.dropout(x)
        x = self.pool(torch.relu(x))

        x = x.view(x.shape[0], -1)

        x = self.fc1(x)
        x = self.dropout(x)
        x = torch.relu(x)

        x = self.fc2(x)

        x = x.squeeze(1)
        return x


C:\Users\marco\AppData\Local\Temp\ipykernel_26604\2551682548.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
C:\Users\marco\AppData\Local\Temp\ipykernel_26604\2551682548.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y, dtype=torch.float32)


In [21]:
def print_semantic_loss(y_pred, y_true):
    print("Mean scaled absolute error", torch.mean(torch.abs(y_pred - y_true) / y_true, dim=0))
    print("Mean absolute error", torch.mean(torch.abs(y_pred - y_true), dim=0))
    print("Scaled absolute error", torch.abs(y_pred - y_true) / y_true)
    print("First few", y_pred[:50], y_true[:5])

In [22]:
model = TradingNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 1000

for epoch in range(EPOCHS):
    for X_batch, Y_batch in tqdm(dataloader):
        X_batch = X_batch
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = criterion(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch} Loss: {loss.item()}")
    
    with torch.no_grad():
        model.eval()
        Y_pred = model(Xtest)
        loss = criterion(Y_pred, Ytest)
        print(f"Test Loss: {loss.item()}")
        print_semantic_loss(Y_pred, Ytest)
        model.train()

100%|██████████| 308/308 [00:05<00:00, 52.73it/s]


Epoch 0 Loss: 4.296921179047786e-05
Test Loss: 0.00010447504610056058
Mean scaled absolute error tensor(inf)
Mean absolute error tensor(0.0070)
Scaled absolute error tensor([-1.0277, -1.0424, -1.2565,  ..., -1.0665, -1.0443, -1.0395])
First few tensor([0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009,
        0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009,
        0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009,
        0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0008]) tensor([-0.0313, -0.0206, -0.0034,  0.0041, -0.0040])


100%|██████████| 308/308 [00:05<00:00, 53.57it/s]


Epoch 1 Loss: 0.0001335359556833282
Test Loss: 0.0001039196431520395
Mean scaled absolute error tensor(inf)
Mean absolute error tensor(0.0070)
Scaled absolute error tensor([-1.0035, -1.0053, -1.0319,  ..., -1.0082, -1.0054, -1.0048])
First few tensor([1.0904e-04, 1.0984e-04, 1.1017e-04, 1.1041e-04, 1.1039e-04, 1.1028e-04,
        1.0855e-04, 1.0779e-04, 1.0778e-04, 1.0862e-04, 1.0985e-04, 1.0925e-04,
        1.0937e-04, 1.0955e-04, 1.0989e-04, 1.0956e-04, 1.0882e-04, 1.0806e-04,
        1.0855e-04, 1.0986e-04, 1.0937e-04, 1.0861e-04, 1.0656e-04, 1.0592e-04,
        1.0594e-04, 1.0520e-04, 1.0461e-04, 1.0329e-04, 1.0348e-04, 1.0463e-04,
        1.0591e-04, 1.0586e-04, 1.0510e-04, 1.0649e-04, 1.0901e-04, 1.1363e-04,
        1.1621e-04, 1.1614e-04, 1.1601e-04, 1.1628e-04, 1.1829e-04, 1.1908e-04,
        1.1655e-04, 1.1127e-04, 1.0674e-04, 1.0418e-04, 1.0372e-04, 1.0202e-04,
        9.9716e-05, 9.8570e-05]) tensor([-0.0313, -0.0206, -0.0034,  0.0041, -0.0040])


100%|██████████| 308/308 [00:05<00:00, 51.90it/s]


...

Epoch 5 Loss: 6.674799806205556e-05
Test Loss: 0.00010403967462480068
Mean scaled absolute error tensor(inf)
Mean absolute error tensor(0.0070)
Scaled absolute error tensor([-1.0151, -1.0230, -1.1375,  ..., -1.0357, -1.0238, -1.0213])
First few tensor([0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005, 0.0005]) tensor([-0.0313, -0.0206, -0.0034,  0.0041, -0.0040])


100%|██████████| 308/308 [00:05<00:00, 52.84it/s]


Epoch 6 Loss: 6.808566104155034e-05
Test Loss: 0.0001039222624967806
Mean scaled absolute error tensor(inf)
Mean absolute error tensor(0.0070)
Scaled absolute error tensor([-1.0025, -1.0037, -1.0224,  ..., -1.0058, -1.0039, -1.0035])
First few tensor([7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05, 7.7242e-05,
        7.7242e-05, 7.7242e-05]) tensor([-0.0313, -0.0206, -0.0034,  0.0041, -0.0040])


 90%|█████████ | 278/308 [00:05<00:00, 50.25it/s]


KeyboardInterrupt: 